<a href="https://colab.research.google.com/github/gvlokesh/Classification/blob/master/anthem_tfidf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
training_desc=['today is a great day', 'it is a very sunny day','Today I have a great plan','plan','great']
training_variable=['x','y','z','m','n']
training_p1=['1','2','3','9','0']
training_p2=['4','5','6','7','8']
df_train = pd.DataFrame(list(zip(training_desc, training_variable,training_p1,training_p2)), 
               columns =['combo', 'var','p1','p2']) 



In [0]:
test_desc=['today is a great ', 'it is  very sunny today']

test_p1=['1','2','3']
test_p2=['4','5','6']
df_test = pd.DataFrame(list(zip(test_desc,test_p1,test_p2)), 
               columns =['combo','p1','p2']) 

In [85]:
df_train

,combo,var,p1,p2
0,today is a great day,x,1,4
1,it is a very sunny day,y,2,5
2,Today I have a great plan,z,3,6
3,plan,m,9,7
4,great,n,0,8


In [86]:
df_test

,combo,p1,p2
0,today is a great,1,4
1,it is very sunny today,2,5


In [87]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []

for i in range(0, len(df_train['combo'])):    
   review = re.sub('[^a-zA-Z0-9]', ' ', df_train['combo'][i]) 
   review = review.lower()  
   review = review.split()   
   ps = PorterStemmer()   
   review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
   review = ' '.join(review   )
   corpus.append(review)
df_train['combo_cleaned']=corpus



#Training

# list of text documents

import joblib


from sklearn.feature_extraction.text import TfidfVectorizer
text =df_train['combo_cleaned']
# create the transform
vectorizer = TfidfVectorizer()
# tokenize and build vocab
vectorizer.fit(text)
# summarize
print(vectorizer.vocabulary_)
print(vectorizer.idf_)
joblib.dump(vectorizer,'vectorizer.pkl')
      


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
{'today': 4, 'great': 1, 'day': 0, 'sunni': 3, 'plan': 2}
[1.69314718 1.40546511 1.69314718 2.09861229 1.69314718]


['vectorizer.pkl']

In [88]:

#Testing cleanup
import joblib
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
vect=joblib.load('vectorizer.pkl')
for i in range(0, len(df_test['combo'])):    
   review = re.sub('[^a-zA-Z0-9]', ' ', df_test['combo'][i]) 
   review = review.lower()  
   review = review.split()   
   ps = PorterStemmer()   
   review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
   review = ' '.join(review   )
   corpus.append(review)
df_test['combo_cleaned']=corpus





[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [89]:
print(df_train,'\n',df_test)

                       combo var p1 p2     combo_cleaned
0       today is a great day   x  1  4   today great day
1     it is a very sunny day   y  2  5         sunni day
2  Today I have a great plan   z  3  6  today great plan
3                       plan   m  9  7              plan
4                      great   n  0  8             great 
                      combo p1 p2 combo_cleaned
0        today is a great   1  4   today great
1  it is  very sunny today  2  5   sunni today


In [90]:
df_train

,combo,var,p1,p2,combo_cleaned
0,today is a great day,x,1,4,today great day
1,it is a very sunny day,y,2,5,sunni day
2,Today I have a great plan,z,3,6,today great plan
3,plan,m,9,7,plan
4,great,n,0,8,great


In [91]:
df_test

,combo,p1,p2,combo_cleaned
0,today is a great,1,4,today great
1,it is very sunny today,2,5,sunni today


In [0]:
def checker(cs):
  from heapq import nlargest 
  listofkeys=[]
  listofvalues=cs
  i=0
  sindex=[]
  sscore=[]
  for i in range (0,len(df_train['combo_cleaned'])):
    listofkeys.append(i)
  di=dict(zip(listofkeys,listofvalues))  
  print(di)
  ThreeHighest = nlargest(3, di, key = di.get) 
  print("Dictionary with 3 highest values:") 
  print("Keys: Values")  
 
  for val in ThreeHighest: 
      print(val, ":", di.get(val)) 
      sindex.append(val)
      sscore.append(di.get(val))
  return(sindex[0],sindex[1],sindex[2],sscore[0],sscore[1],sscore[2])
  

In [93]:
import joblib

from sklearn.metrics.pairwise import cosine_similarity

vect=joblib.load('vectorizer.pkl')
i=0
s1_in=[]
s2_in=[]
s3_in=[]
s1_val=[]
s2_val=[]
s3_val=[]
var1=[]
var2=[]
var3=[]
para1_1=[]
para2_1=[]
para1_2=[]
para2_2=[]
para1_3=[]
para2_3=[]

for i in range(0,len(df_test['combo_cleaned'])):
  j=0
  
  score=[]
  score1=[]
  score2=[]
  score3=[]
  for j in range(0,len(df_train['combo_cleaned'])):
    
    v1=vect.transform([df_test[ 'combo_cleaned'][i]])
    
    v2=vect.transform([df_train['combo_cleaned'][j]])
   
    
    score.append(cosine_similarity (v1, v2))
    score1=[l.tolist() for l in score]
    score2=[item for sublist in score1 for item in sublist]
    score3=[item for sublist in score2 for item in sublist]     
  sin1,sin2,sin3,ss1,ss2,ss3=checker(score3)  
  s1_in.append(sin1)
  var1.append(df_train['var'][sin1])

  para1_1.append(df_train['p1'][sin1])
  para2_1.append(df_train['p2'][sin1])
  
  
  s2_in.append(sin2)
  var2.append(df_train['var'][sin2])
  para1_2.append(df_train['p1'][sin2])
  para2_2.append(df_train['p2'][sin2])
  
  
  s3_in.append(sin3)
  var3.append(df_train['var'][sin3])
  para1_3.append(df_train['p1'][sin3])
  para2_3.append(df_train['p2'][sin3])
  
  s1_val.append(ss1)
  s2_val.append(ss2)
  s3_val.append(ss3)
  

{0: 0.7925411347625899, 1: 0.0, 2: 0.7925411347625899, 3: 0.0, 4: 0.638710577565487}
Dictionary with 3 highest values:
Keys: Values
0 : 0.7925411347625899
2 : 0.7925411347625899
4 : 0.638710577565487
{0: 0.38291340085305664, 1: 0.6057243079292997, 2: 0.38291340085305664, 3: 0.0, 4: 0.0}
Dictionary with 3 highest values:
Keys: Values
1 : 0.6057243079292997
0 : 0.38291340085305664
2 : 0.38291340085305664


In [0]:
df1 = pd.DataFrame(list(zip(s1_in,var1,para1_1,para2_1,s1_val,s2_in,var2,para1_2,para2_2,s2_val,s3_in,var3,para1_3,para2_3,s3_val)), columns=['index1','variable1','p1','p2','Score1','index2','variable2','p1','p2','Score2','index3','variable3','p1','p2','Score3'])

In [0]:
df_final = pd.concat([df_test, df1], axis=1)

In [96]:
df_final

,combo,p1,p2,combo_cleaned,index1,variable1,p1,p2,Score1,index2,variable2,p1,p2,Score2,index3,variable3,p1,p2,Score3
0,today is a great,1,4,today great,0,x,1,4,0.792541,2,z,3,6,0.792541,4,n,0,8,0.638711
1,it is very sunny today,2,5,sunni today,1,y,2,5,0.605724,0,x,1,4,0.382913,2,z,3,6,0.382913


In [97]:
df_train

,combo,var,p1,p2,combo_cleaned
0,today is a great day,x,1,4,today great day
1,it is a very sunny day,y,2,5,sunni day
2,Today I have a great plan,z,3,6,today great plan
3,plan,m,9,7,plan
4,great,n,0,8,great


In [98]:
nl

[[0, 1], [2, 0], [4, 2]]

In [99]:
len(nl)

3

In [0]:
para1=[]
para2=[]
para3=[]
i=j=k=0
for i in range(0,len(s1_in)):
  
  para1.append(df_train['p1'][s1_in[i]])

for j in range(0,len(s2_in)):
 
  para2.append(df_train['p2'][s2_in[i]])
  

    

In [76]:
para1

['1', '2']

In [78]:
para2_3

['8', '6']